In [1]:
import os
import glob
import pandas as pd
import math
import numpy as np
from scipy.misc import imread, imsave, imresize
from natsort import natsorted

import keras

Using TensorFlow backend.
/Users/r3dmaohong/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
path = "data/julia"

img_height, img_width = 32, 32

In [3]:
# save path for images after processed
suffix = "Preproc"
trainDataPath = path + "/train" + suffix
testDataPath = path + "/test" + suffix

if not os.path.exists(trainDataPath):
    os.makedirs(trainDataPath)
if not os.path.exists(testDataPath):
    os.makedirs(testDataPath)
    

for datasetType in ["train","test"]:
    # sort files
    imgFiles = natsorted(glob.glob(path + "/" + datasetType + "/*"))
    
    # save imgs
    imgData = np.zeros((len(imgFiles), img_height, img_width))
    
    for i, imgFilePath in enumerate(imgFiles):
        # True: one channel
        img = imread(imgFilePath, True) 
        
        # image resizing
        imgResized = imresize(img, (img_height, img_width))
        
        imgData[i] = imgResized
        
        # save
        filename = os.path.basename(imgFilePath)
        filenameDotSplit = filename.split(".")
        newFilename = str(int(filenameDotSplit[0])).zfill(5) + "." + filenameDotSplit[-1].lower()
        newFilepath = path + "/" + datasetType + suffix + "/" + newFilename
        imsave(newFilepath, imgResized)
    
    # generate a new dimension for channel
    print(datasetType, "Before: ", imgData.shape)
    imgData = imgData[:,:,:,np.newaxis]
    print(datasetType, "After: ", imgData.shape)
    
    # 進行資料(pixel值)標準化
    imgData = imgData.astype('float32')/255
    
    # 以numpy物件將圖像轉換後的ndarray物件保存在檔案系統中
    np.save(path + "/" + datasetType + suffix + ".npy", imgData)

/Users/r3dmaohong/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/Users/r3dmaohong/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/Users/r3dmaohong/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:33: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


train Before:  (6283, 32, 32)
train After:  (6283, 32, 32, 1)
test Before:  (6220, 32, 32)
test After:  (6220, 32, 32, 1)


In [4]:
# label conversion
# 0-9, a-z, A-Z
# 62

def label2int(ch):
    # Return the Unicode code point for a one-character string.
    asciiVal = ord(ch)
    if(asciiVal<=57): #0-9
        asciiVal-=48
    elif(asciiVal<=90): #A-Z
        asciiVal-=55
    else: #a-z
        asciiVal-=61
    return asciiVal
    
def int2label(i):
    if(i<=9): #0-9
        i+=48
    elif(i<=35): #A-Z
        i+=55
    else: #a-z
        i+=61
    # Return a Unicode string of one character with ordinal i
    return chr(i)

# only retain the label col
y_train = pd.read_csv(path + "/trainLabels.csv").values[:,1]

# one hot encoding
Y_train = np.zeros((y_train.shape[0], 62))

for i in range(y_train.shape[0]):
    Y_train[i][label2int(y_train[i])] = 1

np.save(path + "/" + "labelsPreproc.npy", Y_train)

In [5]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split

batch_size = 128
# 0-9, a-z, A-Z
nb_classes = 62
nb_epoch = 300

img_height, img_width = 32, 32

# training data and lebels after processed
X_train_all = np.load(path+"/trainPreproc.npy")
Y_train_all = np.load(path+"/labelsPreproc.npy")


X_train, X_val, Y_train, Y_val = \
    train_test_split(X_train_all, Y_train_all, test_size=0.25, stratify
                     =np.argmax(Y_train_all, axis=1))

# data augmentation
datagen = ImageDataGenerator(
    rotation_range = 20,
    width_shift_range = 0.15,
    height_shift_range = 0.15,
    shear_range = 0.4,
    zoom_range = 0.3,                    
    channel_shift_range = 0.1)

In [ ]:
model = Sequential()

model.add(Convolution2D(128,(3, 3), padding='same', kernel_initializer='he_normal', activation='relu', 
                        input_shape=(img_height, img_width, 1)))

model.add(Convolution2D(128,(3, 3), padding='same', kernel_initializer='he_normal', activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(256,(3, 3), padding='same', kernel_initializer='he_normal', activation='relu'))
model.add(Convolution2D(256,(3, 3), padding='same', kernel_initializer='he_normal', activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(512,(3, 3), padding='same', kernel_initializer='he_normal', activation='relu'))
model.add(Convolution2D(512,(3, 3), padding='same', kernel_initializer='he_normal', activation='relu'))
model.add(Convolution2D(512,(3, 3), padding='same', kernel_initializer='he_normal', activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(4096, kernel_initializer='he_normal', activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(4096, kernel_initializer='he_normal', activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_classes, kernel_initializer='he_normal', activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 128)       1280      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 256)       295168    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 256)       590080    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 256)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 8, 8, 512)         1180160   
__________

In [ ]:
# use adadelta to be the first part of the training because adamax will be stucked.
print("="*50)
print("AdaDelta")
print("="*50)
model.compile(loss='categorical_crossentropy', 
              optimizer='adadelta',  
              metrics=["accuracy"])

model.fit(X_train, Y_train, batch_size=batch_size,
                    epochs=20, 
                    validation_data=(X_val, Y_val),
                    verbose=1)
print("="*50)
print("AdaMax")
print("="*50)
model.compile(loss='categorical_crossentropy', 
              optimizer='adamax',  
              metrics=["accuracy"])


# save better validation accuracy model
saveBestModel = ModelCheckpoint("best.kerasModelWeights", monitor='val_acc', verbose=1, \
                                save_best_only=True, save_weights_only=True)

# data augmentation
history = model.fit_generator(datagen.flow(X_train, Y_train, batch_size=batch_size),
                    steps_per_epoch=len(X_train)/batch_size,
                    epochs=nb_epoch, 
                    validation_data=(X_val, Y_val),
                    callbacks=[saveBestModel],
                    verbose=1)

# load best validation accuracy model
model.load_weights("best.kerasModelWeights")

# load test data
X_test = np.load(path+"/testPreproc.npy")

# predict
Y_test_pred = model.predict_classes(X_test)

# int to label
vInt2label = np.vectorize(int2label)
Y_test_pred = vInt2label(Y_test_pred) 


# save predict output
np.savetxt(path+"/jular_pred" + ".csv", np.c_[range(6284,len(Y_test_pred)+6284),Y_test_pred],\
           delimiter=',', header = 'ID,Class', comments = '', fmt='%s')

AdaDelta
Train on 4712 samples, validate on 1571 samples
Epoch 1/20
4712/4712 [==============================] - 820s 174ms/step - loss: 14.7871 - acc: 0.0558 - val_loss: 15.2050 - val_acc: 0.0567
Epoch 2/20
4712/4712 [==============================] - 858s 182ms/step - loss: 15.2014 - acc: 0.0569 - val_loss: 15.2050 - val_acc: 0.0567
Epoch 3/20
4712/4712 [==============================] - 860s 182ms/step - loss: 15.2014 - acc: 0.0569 - val_loss: 15.2050 - val_acc: 0.0567
Epoch 4/20
4712/4712 [==============================] - 874s 185ms/step - loss: 15.2014 - acc: 0.0569 - val_loss: 15.2050 - val_acc: 0.0567
Epoch 5/20
4712/4712 [==============================] - 940s 200ms/step - loss: 15.2014 - acc: 0.0569 - val_loss: 15.2050 - val_acc: 0.0567
Epoch 6/20
4712/4712 [==============================] - 849s 180ms/step - loss: 15.2014 - acc: 0.0569 - val_loss: 15.2050 - val_acc: 0.0567
Epoch 7/20
4712/4712 [==============================] - 838s 178ms/step - loss: 15.2014 - acc: 0.0569 -

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()